# Final Project
## Team 25
Santiago Flores - 30119885

Davis Allan - 10016543

Jordan Joorisity - 30117950

Patrick Pickard - 30116807

Joshua Posyluzny - 30118206

# 1. Loading and pre-processing the dataset

### 1.1 Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### 1.2 Load you Kaggle API token
In order to use the Kaggle’s public API, you must first authenticate using an API token. From the site header, click on your user profile picture, then on “My Account” from the dropdown menu. This will take you to your account settings at https://www.kaggle.com/account. Scroll down to the section of the page labelled API:

To create a new token, click on the “Create New API Token” button. This will download a fresh authentication token onto your machine. Reference: (https://www.kaggle.com/docs/api)

With the json token downloaded run the cell below and upload the file

In [ ]:
from google.colab import files
files.upload()

In [ ]:
# Moves your API token key 
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### 1.3 Download and unzip the datasets

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('asl-alphabet.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [ ]:
!kaggle datasets download -d danrasband/asl-alphabet-test

In [ ]:
zip_ref = zipfile.ZipFile('asl-alphabet-test.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

### 1.4 Data display

There are 29 classes in the dataset, one for each letter in the English alphabet, one for space, one for delete and a blank one. Below is an exmaple of each one

In [ ]:
import matplotlib.pyplot as plt

train_folder = 'files/asl_alphabet_train/asl_alphabet_train/'
validation_folder = 'files/asl-alphabet-test/'
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']

plt.figure(figsize=(11,11))
for i in range (0,29):
  plt.subplot(7,7, i+1)
  plt.xticks([])
  plt.yticks([])
  path = train_folder + classes[i] + "/" + classes[i] + "1.jpg"
  img = plt.imread(path)
  plt.imshow(img)
  plt.xlabel(classes[i])

### 1.5 Saving the datasets

In [ ]:
import os
import cv2

def load_images(image_folder, images, labels):
  num = -1
  for folder in os.listdir(image_folder):
    num += 1
    for image in os.listdir(image_folder + "/" + folder):
      temp_img = cv2.imread(image_folder + "/" + folder + "/" + image)
      images.append(temp_img)
      labels.append(folder)

1.5.1 Training data

In [ ]:
train_images = []
train_labels = []

load_images(train_folder, train_images, train_labels)

1.5.2 Test data

In [ ]:
validation_images = []
validation_labels = []

load_images(validation_folder, validation_images, validation_labels)

In [ ]:
  h, w, r = train_images[0].shape
  print('There are', len(train_images), 'images for training the model')
  print(len(train_labels)//len(set(train_labels)), 'images per category')
  print('The shape of each image is', train_images[0].shape)
  print('From which:')
  print('The width is', w)
  print('The height is', h)
  print('And each pixel has a value for each component of RGB for a total of', r)

### 1.6 One Hot Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

train_labels_df = pd.DataFrame(train_labels, columns=['Character'])
labelencoder = LabelEncoder()
onehotencoder = OneHotEncoder()

train_labels_df['label'] = labelencoder.fit_transform(train_labels_df['Character'])
train_labels_onehot = pd.DataFrame(onehotencoder.fit_transform(train_labels_df[['label']]).toarray())

# train_labels_df.drop_duplicates()
train_labels_onehot[:10]

In [ ]:
validation_labels_df = pd.DataFrame(validation_labels, columns=['Character'])

validation_labels_df['label'] = labelencoder.fit_transform(labels_df['Character'])
validation_labels_onehot = pd.DataFrame(onehotencoder.fit_transform(labels_df[['label']]).toarray())

validation_labels_df.drop_duplicates()
# validation_labels_onehot

### 1.7 Train test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels_onehot, 
                                                    test_size=0.2, 
                                                    stratify=train_labels,
                                                    random_state=21)

In [ ]:
print('After the split there are:')
print(len(X_train), 'images on the train split')
print(len(X_test), 'images on the test split')
print('For each category 80% of the images are in the train split and 20% in the test split')